In [165]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
import random
from numpy import linalg
from sklearn.preprocessing import StandardScaler
import time
from sklearn.metrics import confusion_matrix

In [166]:
sentiment_df = pd.read_csv('tweets_sentimientos_trusted.csv',low_memory=False)
sentiment_df.head(5)

,Unnamed: 0,Date,tb_sentiment_agg_like,vader_sentiment_agg_like,tb_sentiment_agg_comment,vader_sentiment_agg_comment,tb_sentiment_agg_retweet,vader_sentiment_agg_retweet,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector
0,0,2015-01-01,-0.270301,0.010241,-0.411497,-0.155643,0.006467,0.115964,-1,-1,0,0,-1,1
1,1,2015-01-02,0.077530,0.121911,0.053342,0.075837,0.197816,0.125320,1,1,1,1,1,1
2,2,2015-01-03,0.084974,0.135777,0.070563,0.169745,-0.006333,0.099781,1,1,0,1,1,1
3,3,2015-01-04,-0.066925,0.197441,-0.111962,0.117920,-0.253550,0.160401,-1,-1,-1,1,1,1
4,4,2015-01-05,0.104477,0.160160,0.089984,0.132301,0.094667,0.149451,1,1,1,1,1,1


In [167]:
positive_treshold = 0.000000001
negative_treshold = -0.000000001

In [168]:
sentiment_df['tb_likes_vector'] = sentiment_df.apply(lambda row: 1 if row['tb_sentiment_agg_like']>positive_treshold else (0 if row['tb_sentiment_agg_like']<negative_treshold else 0),axis=1)
sentiment_df['tb_comments_vector'] = sentiment_df.apply(lambda row: 1 if row['tb_sentiment_agg_comment']>positive_treshold else (0 if row['tb_sentiment_agg_comment']<negative_treshold else 0),axis=1)
sentiment_df['tb_retweets_vector'] = sentiment_df.apply(lambda row: 1 if row['tb_sentiment_agg_retweet']>positive_treshold else (0 if row['tb_sentiment_agg_retweet']<negative_treshold else 0),axis=1)
sentiment_df['vader_likes_vector'] = sentiment_df.apply(lambda row: 1 if row['vader_sentiment_agg_like']>positive_treshold else (0 if row['vader_sentiment_agg_like']<negative_treshold else 0),axis=1)
sentiment_df['vader_comments_vector'] = sentiment_df.apply(lambda row: 1 if row['vader_sentiment_agg_comment']>positive_treshold else (0 if row['vader_sentiment_agg_comment']<negative_treshold else 0),axis=1)
sentiment_df['vader_retweets_vector'] = sentiment_df.apply(lambda row: 1 if row['vader_sentiment_agg_retweet']>positive_treshold else (0 if row['vader_sentiment_agg_retweet']<negative_treshold else 0),axis=1)
sentiment_df.head(5)

,Unnamed: 0,Date,tb_sentiment_agg_like,vader_sentiment_agg_like,tb_sentiment_agg_comment,vader_sentiment_agg_comment,tb_sentiment_agg_retweet,vader_sentiment_agg_retweet,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector
0,0,2015-01-01,-0.270301,0.010241,-0.411497,-0.155643,0.006467,0.115964,0,0,1,1,0,1
1,1,2015-01-02,0.077530,0.121911,0.053342,0.075837,0.197816,0.125320,1,1,1,1,1,1
2,2,2015-01-03,0.084974,0.135777,0.070563,0.169745,-0.006333,0.099781,1,1,0,1,1,1
3,3,2015-01-04,-0.066925,0.197441,-0.111962,0.117920,-0.253550,0.160401,0,0,0,1,1,1
4,4,2015-01-05,0.104477,0.160160,0.089984,0.132301,0.094667,0.149451,1,1,1,1,1,1


In [169]:
#Correlacion entre mercado y tweets 1dia.
market_data = pd.read_csv('dataset_limpio_98.csv',low_memory=False)
market_data.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,rdto,volume_5D,volume_10D,volume_30D,volume_90D,t_rdto_1D,t_rdto_5D,t_rdto_10D,t_rdto_30D,t_rdto_90D
0,2015-01-12,4714.069824,4715.810059,4650.649902,4664.709961,4664.709961,1861960000,-0.836719,0,1,1,0,0,1,0,0,1
1,2015-01-13,4708.740234,4751.339844,4624.279785,4661.500000,4661.500000,2162180000,-0.068814,1,1,1,1,0,1,0,0,1
2,2015-01-14,4610.759766,4655.370117,4595.979980,4639.319824,4639.319824,2073810000,-0.475816,1,1,1,1,0,0,0,0,1
3,2015-01-15,4657.459961,4663.959961,4567.390137,4570.819824,4570.819824,1976260000,-1.476510,0,1,1,1,0,0,0,0,0
4,2015-01-16,4566.379883,4635.819824,4563.109863,4634.379883,4634.379883,1970520000,1.390561,1,1,1,1,1,0,0,0,1


In [170]:
mkt_th_positive = 0.000001
mkt_th_negative = -0.000001

In [171]:
#mkt_th_positive_extreme = 1.2
#mkt_th_negative_extreme = -1.2

#market_data['vector1d'] = market_data.apply(lambda row: 1 if row['rdto']>mkt_th_positive else(0 if row['rdto']<mkt_th_negative else 0),axis=1)

#market_data['vector1d'] = market_data.apply(lambda row: 1 if row['rdto']>mkt_th_positive_extreme else(0 if row['rdto']<mkt_th_negative_extreme else row['vector1d']),axis=1)

#market_data.head(10)

In [172]:
market_vectors = market_data[['Date','rdto','t_rdto_1D','t_rdto_5D','t_rdto_10D','t_rdto_30D','t_rdto_90D','volume_5D',	'volume_10D',	'volume_30D',	'volume_90D']]
market_vectors['rdto_nextday'] = market_vectors['t_rdto_1D'].shift(-1)
market_vectors

C:\Users\Roca\.conda\envs\GPU\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,rdto,t_rdto_1D,t_rdto_5D,t_rdto_10D,t_rdto_30D,t_rdto_90D,volume_5D,volume_10D,volume_30D,volume_90D,rdto_nextday
0,2015-01-12,-0.836719,0,1,0,0,1,0,1,1,0,0.0
1,2015-01-13,-0.068814,0,1,0,0,1,1,1,1,1,0.0
2,2015-01-14,-0.475816,0,0,0,0,1,1,1,1,1,0.0
3,2015-01-15,-1.476510,0,0,0,0,0,0,1,1,1,1.0
4,2015-01-16,1.390561,1,0,0,0,1,1,1,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1219,2019-12-23,0.231827,1,1,1,1,1,0,0,0,1,1.0
1220,2019-12-24,0.080816,1,1,1,1,1,0,0,0,0,1.0
1221,2019-12-26,0.776396,1,1,1,1,1,0,0,0,0,0.0
1222,2019-12-27,-0.174782,0,1,1,1,1,0,0,0,0,0.0


In [173]:
#Juntamos con sentimientos.
sentiment_df.rename(columns={'post_date_utc':'Date'}, inplace = True)
print(sentiment_df.columns)
joined_df = pd.merge(sentiment_df,market_vectors,how='left',on=['Date'])


Index(['Unnamed: 0', 'Date', 'tb_sentiment_agg_like',
       'vader_sentiment_agg_like', 'tb_sentiment_agg_comment',
       'vader_sentiment_agg_comment', 'tb_sentiment_agg_retweet',
       'vader_sentiment_agg_retweet', 'tb_likes_vector', 'tb_comments_vector',
       'tb_retweets_vector', 'vader_likes_vector', 'vader_comments_vector',
       'vader_retweets_vector'],
      dtype='object')


In [174]:
joined_df

,Unnamed: 0,Date,tb_sentiment_agg_like,vader_sentiment_agg_like,tb_sentiment_agg_comment,vader_sentiment_agg_comment,tb_sentiment_agg_retweet,vader_sentiment_agg_retweet,tb_likes_vector,tb_comments_vector,...,t_rdto_1D,t_rdto_5D,t_rdto_10D,t_rdto_30D,t_rdto_90D,volume_5D,volume_10D,volume_30D,volume_90D,rdto_nextday
0,0,2015-01-01,-0.270301,0.010241,-0.411497,-0.155643,0.006467,0.115964,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2015-01-02,0.077530,0.121911,0.053342,0.075837,0.197816,0.125320,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2015-01-03,0.084974,0.135777,0.070563,0.169745,-0.006333,0.099781,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2015-01-04,-0.066925,0.197441,-0.111962,0.117920,-0.253550,0.160401,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2015-01-05,0.104477,0.160160,0.089984,0.132301,0.094667,0.149451,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,1821,2019-12-27,0.048672,0.144104,0.043899,0.123508,0.116598,0.254251,1,1,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1822,1822,2019-12-28,0.101040,0.204908,0.067188,0.154332,0.097260,0.179383,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1823,1823,2019-12-29,0.059166,0.099699,0.068911,0.090600,0.120346,0.165431,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1824,1824,2019-12-30,0.108990,0.162796,0.093819,0.098175,0.281425,0.452046,1,1,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,NaN


In [175]:
#Botamos los NaN, se van en 1 de tres escenarios, es fin de semana, es feriado, o fue un día de outlier.
joined_df = joined_df[joined_df['rdto_nextday'].notna()]
joined_df

,Unnamed: 0,Date,tb_sentiment_agg_like,vader_sentiment_agg_like,tb_sentiment_agg_comment,vader_sentiment_agg_comment,tb_sentiment_agg_retweet,vader_sentiment_agg_retweet,tb_likes_vector,tb_comments_vector,...,t_rdto_1D,t_rdto_5D,t_rdto_10D,t_rdto_30D,t_rdto_90D,volume_5D,volume_10D,volume_30D,volume_90D,rdto_nextday
11,11,2015-01-12,0.097960,0.116047,0.035764,0.050992,0.076754,0.114367,1,1,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
12,12,2015-01-13,0.099349,0.092123,0.069313,0.030635,0.081086,0.101457,1,1,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
13,13,2015-01-14,0.063843,0.075388,0.043298,0.038493,0.055820,0.016293,1,1,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
14,14,2015-01-15,0.082466,0.088818,0.066406,0.064309,0.082109,0.063026,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
15,15,2015-01-16,0.054490,0.117908,0.032463,0.057512,0.167061,0.057487,1,1,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,1813,2019-12-19,0.084212,0.225914,0.084647,0.012938,0.099074,0.190347,1,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1817,1817,2019-12-23,0.095159,0.202943,0.086354,0.138893,0.104797,0.217780,1,1,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
1818,1818,2019-12-24,0.097582,0.184620,0.098713,0.281355,0.107862,0.248562,1,1,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1820,1820,2019-12-26,0.122950,0.143449,0.100937,0.132633,0.122595,0.201793,1,1,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [176]:
correlation_df = joined_df.drop(columns=['Unnamed: 0','Date','tb_sentiment_agg_like',	'vader_sentiment_agg_like',	'tb_sentiment_agg_comment',	'vader_sentiment_agg_comment',	'tb_sentiment_agg_retweet',	'vader_sentiment_agg_retweet','rdto'])
correlation_df.head(5)

,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,t_rdto_1D,t_rdto_5D,t_rdto_10D,t_rdto_30D,t_rdto_90D,volume_5D,volume_10D,volume_30D,volume_90D,rdto_nextday
11,1,1,1,1,1,1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
12,1,1,1,1,1,1,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
13,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
14,1,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
15,1,1,1,1,1,1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [177]:
correlation_df.corr()

,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,t_rdto_1D,t_rdto_5D,t_rdto_10D,t_rdto_30D,t_rdto_90D,volume_5D,volume_10D,volume_30D,volume_90D,rdto_nextday
tb_likes_vector,1.000000,0.175193,0.326401,0.321634,0.041998,0.184224,-0.007003,0.059105,0.045625,0.022681,0.040637,-0.003641,0.014238,0.031538,0.030517,0.029596
tb_comments_vector,0.175193,1.000000,0.087618,0.047286,0.295489,0.089388,0.012385,0.006794,-0.039903,-0.008142,-0.011903,0.012613,0.062108,0.044545,0.026807,-0.004333
tb_retweets_vector,0.326401,0.087618,1.000000,0.128142,0.133580,0.402102,0.010070,0.039279,0.048197,0.031888,0.001150,0.003680,0.044162,0.042572,0.071519,-0.000253
vader_likes_vector,0.321634,0.047286,0.128142,1.000000,0.203559,0.348978,-0.013104,0.014606,0.009337,-0.021303,0.000514,0.012521,0.024120,0.046882,0.057389,0.011159
vader_comments_vector,0.041998,0.295489,0.133580,0.203559,1.000000,0.263059,0.010070,-0.002860,-0.026552,0.031888,0.001150,0.054945,0.064670,0.073349,0.071519,-0.020901
vader_retweets_vector,0.184224,0.089388,0.402102,0.348978,0.263059,1.000000,0.010357,0.036826,-0.012447,-0.025194,-0.005389,0.009166,0.008404,0.013527,0.018277,-0.056741
t_rdto_1D,-0.007003,0.012385,0.010070,-0.013104,0.010070,0.010357,1.000000,0.286226,0.194446,0.098973,0.053263,-0.098090,-0.110892,-0.096520,-0.083598,-0.042881
t_rdto_5D,0.059105,0.006794,0.039279,0.014606,-0.002860,0.036826,0.286226,1.000000,0.517607,0.220805,0.072593,-0.023209,-0.116687,-0.124492,-0.130253,0.001535
t_rdto_10D,0.045625,-0.039903,0.048197,0.009337,-0.026552,-0.012447,0.194446,0.517607,1.000000,0.414521,0.116986,0.019467,0.003370,-0.089329,-0.118470,-0.018548
t_rdto_30D,0.022681,-0.008142,0.031888,-0.021303,0.031888,-0.025194,0.098973,0.220805,0.414521,1.000000,0.374461,0.005313,-0.025711,-0.093501,-0.133446,-0.017601


In [178]:


def PCA(data):
  x_standard = StandardScaler().fit_transform(data)

  s_cov = np.cov(x_standard.T)
  eigenval, eigenvec = np.linalg.eig(s_cov)
  information = eigenval/sum(eigenval)
  print(information*100)

In [179]:
PCA(correlation_df)

[18.62579757 13.06036025 11.75328259  1.2817602   1.63387537  7.07106336
  7.15313538  6.41922639  5.70819624  5.46053024  5.37630553  2.57336695
  2.75680875  3.41672171  3.78456429  3.92500518]


In [180]:
PCA(correlation_df.drop(columns=['rdto_nextday']))

[19.86751538 13.92503496 12.53606615  7.49735279  7.62305115  1.37311666
  1.74716352  6.19474106  5.87600012  5.77825852  2.75299656  2.94339015
  3.66170645  4.03687488  4.18673167]


In [181]:
dataset = joined_df.copy()
dataset = dataset.drop(columns=['Unnamed: 0','Date','tb_sentiment_agg_like',	'vader_sentiment_agg_like',	'tb_sentiment_agg_comment',	'vader_sentiment_agg_comment',	'tb_sentiment_agg_retweet',	'vader_sentiment_agg_retweet'])
dataset.reset_index(inplace=True)
dataset.drop(columns=['index'],inplace=True)
dataset

,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,rdto,t_rdto_1D,t_rdto_5D,t_rdto_10D,t_rdto_30D,t_rdto_90D,volume_5D,volume_10D,volume_30D,volume_90D,rdto_nextday
0,1,1,1,1,1,1,-0.836719,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
1,1,1,1,1,1,1,-0.068814,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
2,1,1,1,1,1,1,-0.475816,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
3,1,1,1,1,1,1,-1.476510,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
4,1,1,1,1,1,1,1.390561,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218,1,1,1,1,1,1,0.673779,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1219,1,1,1,1,1,1,0.231827,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
1220,1,1,1,1,1,1,0.080816,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1221,1,1,1,1,1,1,0.776396,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [184]:
#Time Series Lags Generator

dataset_train_test = dataset.copy()
#dataset_train_test = dataset_train_test[0:len(dataset_train_test)-60]
#dataset_validation = dataset_train_test[len(dataset_train_test)-60-60:len(dataset_train_test)]

def ts_generator(lag,X,Y):
  new_X = X.copy()

  for i in range(lag):
    df_lag = X.copy()
    df_lag = df_lag.shift(lag)
    df_lag = df_lag.add_suffix("_lag_"+str(i+1))
    new_X = pd.merge(new_X,df_lag,left_index=True,right_index=True)
  
  new_X = new_X[new_X.columns.drop(list(new_X.filter(regex='rdto_lag')))]

  new_X = new_X.dropna()

  new_Y = Y.iloc[lag:]

  return new_X,new_Y


In [183]:
0.5208093220338983
18.722112498900923
17.668370365534894

17.668370365534894

In [20]:
#Lag 60
0.5588669527896989
20.77148140088323
19.740285668145603

#Lag 45
0.5197584745762721
17.949189384726814
16.90681519920231

#Lag 30
0.5340794979079494
14.586597941477434
13.632730630797399

#Lag 20
0.5289875518672196
20.570987753502305
19.515140417941367

#Lag 15
0.5405082644628111
19.794697371349102
18.854576075107268

#Lag 10
0.5311810699588476
16.363515853468968
15.35552118870345

15.35552118870345

In [487]:
def walk_forward_SVM_model(lag,start,train,test,X,Y,C):

    new_X,new_Y = ts_generator(lag,X,Y)

    #70d Training, 30d testing.

    max_length = int(((len(new_X)-train)/test)-test)

    periods = max_length #

    #Primeros 30 días

    vec_acc_training = []
    vec_acc_testing = []
    vec_rend_training = []
    vec_rend_testing = []

    vec_rend_real_tr = []
    vec_rend_real_ts = []
    vec_pred_tr = []
    vec_pred_ts = []

    vec_dv_train = []

    ann_rend_training = 0
    ann_rend_testing = 0

    rend_train_returns = []


    for i in range(periods):

        X_train = new_X[start:start+train+1]
        X_test = new_X[start+train:start+train+test+1]

        y_train = new_Y[start:start+train+1]
        y_test = new_Y[start+train:start+train+test+1]
        
        X_train.reset_index(inplace=True)
        y_train = y_train.reset_index()
        y_train = y_train['rdto_nextday']
        X_test.reset_index(inplace=True)
        y_test = y_test.reset_index()
        y_test = y_test['rdto_nextday']

        X_train_return = X_train[['rdto']]
        X_train_return = X_train_return.iloc[1:]
        X_train_return.reset_index(inplace=True)
        
        X_test_return = X_test[['rdto']]
        X_test_return = X_test_return.iloc[1:]
        X_test_return.reset_index(inplace=True)
        
        X_train = X_train.iloc[:-1,:]
        y_train = y_train.iloc[:-1]
        
        X_test = X_test.iloc[:-1,:]
        y_test = y_test.iloc[:-1]

        X_train = X_train.drop(columns=['rdto'])
        X_test = X_test.drop(columns=['rdto'])
        

        acc_tr, acc_ts, y_pred_test, y_pred_train, conf_train,conf_test = SVM_model_wf(X_train,y_train,X_test,y_test,C)

        vec_acc_training.append(round(acc_tr,2))
        vec_acc_testing.append(round(acc_ts,2))

        rend_train = X_train_return['rdto']/100 * y_pred_train
        rend_train = rend_train.tolist()  


        dv_train = 1
        dum_dv = []
        
        for y in range(len(rend_train)):
            dv_train = dv_train * (1+(rend_train[y]))
        dum_dv.append(dv_train)

        dv_train = (dv_train**(1/len(rend_train)))-1
        vec_dv_train.append(dum_dv)


        rend_test = X_test_return['rdto']/100 * y_pred_test
        rend_test = rend_test.tolist()

        rend_train_returns.append(X_test_return['rdto']/100)

        dv_test = 1
        for y in range(len(rend_test)):
            dv_test = dv_test * (1+(rend_test[y]))
        dv_test = (dv_test**(1/len(rend_test)))-1


        vec_rend_training.append(round(dv_train,6))
        vec_rend_testing.append(round(dv_test,6))

        start += test

        vec_rend_real_tr.append(X_train_return['rdto'].tolist())
        vec_rend_real_ts.append(rend_test)
        vec_pred_tr.append(y_pred_train)
        vec_pred_ts.append(y_pred_test)

    dv_train = 1
    for y in range(len(vec_rend_training)):
        dv_train = dv_train * (1+(vec_rend_training[y]))
    dv_train = (dv_train**(1/len(vec_rend_training)))
    ann_return_train = (dv_train**240)-1


    dv_test = 1
    for y in range(len(vec_rend_testing)):
        dv_test = dv_test * (1+(vec_rend_testing[y]))
    dv_test = (dv_test**(1/len(vec_rend_testing)))
    ann_return_test = (dv_test**240)-1

    """
    print("Vector Training Acc: ",vec_acc_training)
    print("Vector Testing Acc: ",vec_acc_testing)
    print("Vector Returns Training: ",vec_rend_training)
    print("Vector Returns Testing: ",vec_rend_testing)
    print("")
    print("Training Avg Accuracy: ",round(sum(vec_acc_training)/len(vec_acc_training),2))
    print("")
    print("Training Annual Return", round(ann_return_train,3)*100)
    print("")
    print("Testing Avg Accuracy: ",round(sum(vec_acc_testing)/len(vec_acc_testing),2))
    print("")
    print("Testing Annual Return", round(ann_return_test,3)*100)
    """

    tr_acc = round(sum(vec_acc_training)/len(vec_acc_training),3)
    ts_acc = round(sum(vec_acc_testing)/len(vec_acc_testing),3)
    tr_ret = round(ann_return_train,3)*100
    ts_ret = round(ann_return_test,3)*100

    rend_train_returns.append(X_test_return['rdto']/100)

    return  tr_acc,ts_acc,tr_ret,ts_ret,vec_rend_real_tr,vec_rend_real_ts,vec_pred_tr,vec_pred_ts,vec_dv_train,conf_train,conf_test
          

In [508]:
def SVM_model_wf(X_train,y_train,X_test,y_test, C):
  
    model = svm.SVC(C, kernel='rbf', gamma='auto')
    model = model.fit(X_train,y_train)
    y_pred_train = model.predict(X_train)
    y_pred = model.predict(X_test)

    acc_test = accuracy_score(y_test, y_pred)
    acc_train = accuracy_score(y_train, y_pred_train)

    conf_train = confusion_matrix(y_train, y_pred_train)
    conf_test = confusion_matrix(y_test, y_pred)

    def f(x):
        y = x
        if(x == 0): y=-1 
        else: y=1
        return y

    y_pred = ([f(x) for x in y_pred])
    y_pred_train = ([f(x) for x in y_pred_train])
    
    return acc_train, acc_test,y_pred,y_pred_train,conf_train,conf_test

In [432]:
lag = 3
start = 0
train = 300
test = 20

list_lag = []
list_tr_acc = []
list_ts_acc = []
list_tr_ret = []
list_ts_ret = []

X = dataset_train_test.copy()
X = X.drop(columns=['rdto_nextday']) #No quitar rdto nominal.
Y = dataset_train_test['rdto_nextday']

t = time.time()
tr_acc,ts_acc,tr_ret,ts_ret,vec_rend_real_tr,vec_rend_real_ts,vec_pred_tr,vec_pred_ts,vec_dv_train,conf_train,conf_test = walk_forward_SVM_model(lag,start,train,test,X,Y)

list_lag.append(lag)
list_tr_acc.append(tr_acc)
list_ts_acc.append(ts_acc)
list_tr_ret.append(round(tr_ret,2))
list_ts_ret.append(round(ts_ret,2))


frame = {'LAG':list_lag,'Training Accuracy':list_tr_acc,'Testing Accuracy':list_ts_acc,"Training Return":list_tr_ret,"Testing Return":list_ts_ret}
results = pd.DataFrame(frame)
f_time = round((time.time()-t)/60,2)
results

,LAG,Training Accuracy,Testing Accuracy,Training Return,Testing Return
0,3,0.638,0.575,52.8,25.5


In [433]:
print("Tiempo entrenando el modelo: "+str(f_time))

Tiempo entrenando el modelo: 0.01


In [500]:
import warnings
warnings.filterwarnings('ignore')

lag = 1
start = 0
train = 180
test = 5
C = 0.3

list_lag = []
list_tr_acc = []
list_ts_acc = []
list_tr_ret = []
list_ts_ret = []

list_conf_train = []
list_conf_test = []

X = dataset_train_test.copy()
X = X.drop(columns=['rdto_nextday']) #No quitar rdto nominal.
Y = dataset_train_test['rdto_nextday']

t = time.time()
for i in range(1,lag+1,1):
  tr_acc,ts_acc,tr_ret,ts_ret,vec_rend_real_tr,vec_rend_real_ts,vec_pred_tr,vec_pred_ts,vec_dv_train,conf_train,conf_test = walk_forward_SVM_model(i,start,train,test,X,Y,C)
  list_lag.append(i)
  list_tr_acc.append(tr_acc)
  list_ts_acc.append(ts_acc)
  list_tr_ret.append(round(tr_ret,2))
  list_ts_ret.append(round(ts_ret,2))
    
f_time = round((time.time()-t)/60,2)

frame = {'LAG':list_lag,'Training Accuracy':list_tr_acc,'Testing Accuracy':list_ts_acc,"Training Return":list_tr_ret,"Testing Return":list_ts_ret}
results = pd.DataFrame(frame)
results

,LAG,Training Accuracy,Testing Accuracy,Training Return,Testing Return
0,1,0.633,0.569,58.2,22.4


In [509]:
import warnings
warnings.filterwarnings('ignore')

lag = 1
start = 0
train = 180
test = 5
C = 0.3

C = [0.1,0.5,1.5]

for z in range(1,15,1):
    C = z/10
    list_lag = []
    list_tr_acc = []
    list_ts_acc = []
    list_tr_ret = []
    list_ts_ret = []

    list_conf_train = []
    list_conf_test = []

    X = dataset_train_test.copy()
    X = X.drop(columns=['rdto_nextday']) #No quitar rdto nominal.
    Y = dataset_train_test['rdto_nextday']

    t = time.time()
    for i in range(1,lag+1,1):
      tr_acc,ts_acc,tr_ret,ts_ret,vec_rend_real_tr,vec_rend_real_ts,vec_pred_tr,vec_pred_ts,vec_dv_train,conf_train,conf_test = walk_forward_SVM_model(i,start,train,test,X,Y,C)
      list_lag.append(i)
      list_tr_acc.append(tr_acc)
      list_ts_acc.append(ts_acc)
      list_tr_ret.append(round(tr_ret,2))
      list_ts_ret.append(round(ts_ret,2))

    f_time = round((time.time()-t)/60,2)

    frame = {'LAG':list_lag,'C':C,'Tr Accuracy':list_tr_acc,'Ts Accuracy':list_ts_acc,"Tr Return":list_tr_ret,"Ts Return":list_ts_ret}
    results = pd.DataFrame(frame)
    print(results)

   LAG    C  Tr Accuracy  Ts Accuracy  Tr Return  Ts Return
0    1  0.1        0.565        0.561       19.3       20.3
   LAG    C  Tr Accuracy  Ts Accuracy  Tr Return  Ts Return
0    1  0.2        0.588        0.562       31.0       20.7
   LAG    C  Tr Accuracy  Ts Accuracy  Tr Return  Ts Return
0    1  0.3        0.633        0.569       58.2       22.4
   LAG    C  Tr Accuracy  Ts Accuracy  Tr Return  Ts Return
0    1  0.4        0.667        0.567       76.9       20.5
   LAG    C  Tr Accuracy  Ts Accuracy  Tr Return  Ts Return
0    1  0.5        0.689        0.554       94.0       13.8
   LAG    C  Tr Accuracy  Ts Accuracy  Tr Return  Ts Return
0    1  0.6        0.706        0.559      103.2       16.6
   LAG    C  Tr Accuracy  Ts Accuracy  Tr Return  Ts Return
0    1  0.7        0.713        0.567      107.5       24.1
   LAG    C  Tr Accuracy  Ts Accuracy  Tr Return  Ts Return
0    1  0.8        0.722        0.559      112.5       19.0
   LAG    C  Tr Accuracy  Ts Accuracy  T

In [496]:
results

,LAG,Training Accuracy,Testing Accuracy,Training Return,Testing Return
0,1,0.722,0.559,112.5,19.0


In [ ]:
results

In [455]:
print(results['Training Return'].mean())
print(results['Testing Return'].mean())
print(results['Training Accuracy'].mean())
print(results['Testing Accuracy'].mean())

71.25000000000001
8.35
0.6767333333333334
0.5379


In [451]:
print(results['Training Return'].mean())
print(results['Testing Return'].mean())
print(results['Training Accuracy'].mean())
print(results['Testing Accuracy'].mean())

71.09333333333333
10.659999999999998
0.6766
0.5562333333333334


In [447]:
results 1.0

,LAG,Training Accuracy,Testing Accuracy,Training Return,Testing Return
0,1,0.725,0.513,109.8,3.8
1,2,0.709,0.519,97.4,3.7
2,3,0.702,0.509,90.0,0.9
3,4,0.699,0.506,88.7,-1.3
4,5,0.697,0.469,92.4,-5.5
5,6,0.668,0.518,68.8,5.0
6,7,0.673,0.501,69.6,-1.1
7,8,0.670,0.510,61.7,-1.5
8,9,0.676,0.513,64.8,-4.7
9,10,0.673,0.524,63.4,3.6


In [448]:
print(results['Testing Return'].mean())
print(results['Training Return'].mean())

2.8133333333333326
68.80666666666666


In [439]:
results 0.5

,LAG,Training Accuracy,Testing Accuracy,Training Return,Testing Return
0,1,0.681,0.518,81.9,4.4
1,2,0.651,0.515,61.3,1.9
2,3,0.647,0.513,57.4,-1.3
3,4,0.657,0.505,68.5,-1.2
4,5,0.644,0.508,61.7,-0.3
5,6,0.628,0.514,50.6,1.4
6,7,0.643,0.514,54.9,3.2
7,8,0.642,0.523,53.1,8.5
8,9,0.635,0.517,50.7,0.1
9,10,0.630,0.536,42.3,8.5


In [440]:
print(results['Testing Return'].mean())
print(results['Training Return'].mean())

4.536666666666666
50.403333333333336


In [437]:
results 0.3

,LAG,Training Accuracy,Testing Accuracy,Training Return,Testing Return
0,1,0.644,0.538,57.8,10.0
1,2,0.624,0.532,50.1,8.7
2,3,0.616,0.509,42.9,-2.9
3,4,0.636,0.508,52.5,-1.4
4,5,0.621,0.515,48.4,2.3
5,6,0.618,0.524,41.8,4.2
6,7,0.629,0.509,47.7,1.8
7,8,0.624,0.521,45.4,6.9
8,9,0.617,0.510,39.3,-1.7
9,10,0.605,0.535,32.6,6.6


In [435]:
print(results['Testing Return'].mean())
print(results['Training Return'].mean())

6.203333333333334
41.96000000000001


In [424]:
results reg 0.1

,LAG,Training Accuracy,Testing Accuracy,Training Return,Testing Return
0,1,0.612,0.533,41.4,10.2
1,2,0.600,0.532,36.7,8.6
2,3,0.604,0.512,38.2,-2.9
3,4,0.620,0.504,45.6,-2.1
4,5,0.603,0.515,39.4,2.2
5,6,0.593,0.528,32.9,6.1
6,7,0.612,0.509,39.0,1.8
7,8,0.608,0.515,36.3,5.3
8,9,0.604,0.508,33.7,0.7
9,10,0.594,0.533,28.7,8.7


In [425]:
print(results['Testing Return'].mean())
print(results['Training Return'].mean())

6.659999999999999
37.013333333333335


### Super Modelo

In [154]:
list_lag = []
list_tr_acc = []
list_ts_acc = []
list_tr_ret = []
list_ts_ret = []
list_tr_set = []
list_ts_set = []

list_conf_train = []
list_conf_test = []

#training = [30,40,50,60,70,80,100,120,140,160,180,200,250,300]
#testing = [1,3,5,10,15]

X = dataset_train_test.copy()
X = X.drop(columns=['rdto_nextday']) #No quitar rdto nominal.
Y = dataset_train_test['rdto_nextday']

lag = 30
start = 0
training = [30,40,50,60,70,80,90,100,120,140,160,180,200,250,300,350,400]
testing = [1,5,10,15,20]


t = time.time()
for x in range(len(training)):
    train = training[x]
    
    for y in range(len(testing)):
        test = testing[y]
        
        for i in range(1,lag+1,1):
          tr_acc,ts_acc,tr_ret,ts_ret,vec_rend_real_tr,vec_rend_real_ts,vec_pred_tr,vec_pred_ts,vec_dv_train,conf_train,conf_test = walk_forward_SVM_model(i,start,train,test,X,Y)
          list_lag.append(i)
          list_tr_acc.append(tr_acc)
          list_ts_acc.append(ts_acc)
          list_tr_ret.append(round(tr_ret,2))
          list_ts_ret.append(round(ts_ret,2))
          list_tr_set.append(train)
          list_ts_set.append(test)
          list_conf_train.append(conf_train)
          list_conf_test.append(conf_test)
    
        print("Done with: ",x,", ",y)
    
f_time = round((time.time()-t)/60,2)

frame = {'LAG':list_lag,'Train Days':list_tr_set,'Test Days':list_ts_set,'Training Accuracy':list_tr_acc,'Testing Accuracy':list_ts_acc,"Training Return":list_tr_ret,"Testing Return":list_ts_ret}
results = pd.DataFrame(frame)

Done with:  0 ,  4
Done with:  1 ,  4
Done with:  2 ,  4
Done with:  3 ,  4
Done with:  4 ,  4
Done with:  5 ,  4
Done with:  6 ,  4
Done with:  7 ,  4
Done with:  8 ,  4
Done with:  9 ,  4
Done with:  10 ,  4
Done with:  11 ,  4
Done with:  12 ,  4
Done with:  13 ,  4
Done with:  14 ,  4
Done with:  15 ,  4
Done with:  16 ,  4


In [514]:
import warnings
warnings.filterwarnings('ignore')

list_lag = []
list_tr_acc = []
list_ts_acc = []
list_tr_ret = []
list_ts_ret = []
list_tr_set = []
list_ts_set = []

list_conf_train = []
list_conf_test = []
list_C = []

#training = [30,40,50,60,70,80,100,120,140,160,180,200,250,300]
#testing = [1,3,5,10,15,20]

X = dataset_train_test.copy()
X = X.drop(columns=['rdto_nextday']) #No quitar rdto nominal.
Y = dataset_train_test['rdto_nextday']

lag = 30
start = 0
training = [30,40,50,60,70,80,90,100,120,140,160,180,200,250,300,350,400]
testing = [20,15,10,5,1]
C = 0.5


t = time.time()
for x in range(len(training)):
    train = training[x]

    w = time.time()

    for y in range(len(testing)):
        test = testing[y]

        for i in range(1,lag+1,1):
          tr_acc,ts_acc,tr_ret,ts_ret,vec_rend_real_tr,vec_rend_real_ts,vec_pred_tr,vec_pred_ts,vec_dv_train,conf_train,conf_test = walk_forward_SVM_model(i,start,train,test,X,Y,C)
          list_lag.append(i)
          list_tr_acc.append(tr_acc)
          list_ts_acc.append(ts_acc)
          list_tr_ret.append(round(tr_ret,2))
          list_ts_ret.append(round(ts_ret,2))
          list_tr_set.append(train)
          list_ts_set.append(test)
          list_conf_train.append(conf_train)
          list_conf_test.append(conf_test)
          list_C.append(C)


        print("Done with: ",x,", ",y)

    w_final = round((time.time()-w)/60,2)
    print("Terminamos :"+str(testing[y])+"en: "+str(w_final))
f_time = round((time.time()-t)/60,2)

frame = {'LAG':list_lag,'C':C,'Train Days':list_tr_set,'Test Days':list_ts_set,'Training Accuracy':list_tr_acc,'Testing Accuracy':list_ts_acc,"Training Return":list_tr_ret,"Testing Return":list_ts_ret}
results = pd.DataFrame(frame)

Done with:  0 ,  0
Done with:  0 ,  1
Done with:  0 ,  2
Done with:  0 ,  3
Done with:  0 ,  4
Terminamos :1en: 13.53
Done with:  1 ,  0
Done with:  1 ,  1
Done with:  1 ,  2
Done with:  1 ,  3
Done with:  1 ,  4
Terminamos :1en: 13.47
Done with:  2 ,  0
Done with:  2 ,  1
Done with:  2 ,  2
Done with:  2 ,  3
Done with:  2 ,  4
Terminamos :1en: 13.83
Done with:  3 ,  0
Done with:  3 ,  1
Done with:  3 ,  2
Done with:  3 ,  3
Done with:  3 ,  4
Terminamos :1en: 14.13
Done with:  4 ,  0
Done with:  4 ,  1
Done with:  4 ,  2
Done with:  4 ,  3
Done with:  4 ,  4
Terminamos :1en: 11.6
Done with:  5 ,  0
Done with:  5 ,  1
Done with:  5 ,  2
Done with:  5 ,  3
Done with:  5 ,  4
Terminamos :1en: 10.77
Done with:  6 ,  0
Done with:  6 ,  1
Done with:  6 ,  2
Done with:  6 ,  3
Done with:  6 ,  4
Terminamos :1en: 11.01
Done with:  7 ,  0
Done with:  7 ,  1
Done with:  7 ,  2
Done with:  7 ,  3
Done with:  7 ,  4
Terminamos :1en: 11.32
Done with:  8 ,  0
Done with:  8 ,  1
Done with:  8 ,  2


In [515]:
import winsound
duration = 1000
freq = 400
winsound.Beep(freq,duration)
print("Tiempo entrenando el modelo: "+str(f_time))

Tiempo entrenando el modelo: 270.09


In [516]:
conf_list_training = pd.DataFrame({'matrix training':list_conf_train})
conf_list_training.to_pickle("conf_list_training_c0.5.pkl")

In [517]:
results.to_excel("results_c0.5.xlsx")

### Super Modelo Validacion

In [29]:
lag = 28
start = 0
train = 30
test = 1

list_lag = []
list_tr_acc = []
list_ts_acc = []
list_tr_ret = []
list_ts_ret = []

list_conf_train = []
list_conf_test = []

X = dataset_validation.copy()
X = X.drop(columns=['rdto_nextday']) #No quitar rdto nominal.
Y = dataset_validation['rdto_nextday']

t = time.time()
for i in range(1,lag+1,1):
  tr_acc,ts_acc,tr_ret,ts_ret,vec_rend_real_tr,vec_rend_real_ts,vec_pred_tr,vec_pred_ts,vec_dv_train,conf_train,conf_test = walk_forward_SVM_model(i,start,train,test,X,Y)
  list_lag.append(i)
  list_tr_acc.append(tr_acc)
  list_ts_acc.append(ts_acc)
  list_tr_ret.append(round(tr_ret,2))
  list_ts_ret.append(round(ts_ret,2))
  print("work : ",i)
f_time = round((time.time()-t)/60,2)

frame = {'LAG':list_lag,'Training Accuracy':list_tr_acc,'Testing Accuracy':list_ts_acc,"Training Return":list_tr_ret,"Testing Return":list_ts_ret}
results = pd.DataFrame(frame)
results

work :  1
work :  2
work :  3
work :  4
work :  5
work :  6
work :  7
work :  8
work :  9
work :  10
work :  11
work :  12
work :  13
work :  14
work :  15
work :  16
work :  17
work :  18
work :  19
work :  20
work :  21
work :  22
work :  23
work :  24
work :  25
work :  26
work :  27
work :  28


,LAG,Training Accuracy,Testing Accuracy,Training Return,Testing Return
0,1,0.590,0.500,-24.2,53.7
1,2,0.596,0.556,-10.6,11.8
2,3,0.561,0.538,-18.9,22.1
3,4,0.617,0.520,-9.8,73.6
4,5,0.568,0.542,-16.6,-10.9
5,6,0.560,0.522,-16.8,1.8
6,7,0.583,0.545,-23.7,-22.5
7,8,0.583,0.476,-20.3,42.7
8,9,0.560,0.550,-20.2,-5.4
9,10,0.559,0.526,-19.3,-7.2


In [164]:
f_time

0.08

In [35]:
results['Testing Accuracy'].mean()

0.5938928571428571

In [34]:
results['Training Accuracy'].mean()

0.5733571428571429

In [163]:
lag = 30
start = 0
train = 30
test = 1


list_acc = []
list_yreal = []
list_ypred = []

votes = []

X_ds = dataset_validation.copy()
X_ds = X_ds.drop(columns=['rdto_nextday']) #No quitar rdto nominal.
Y_ds = dataset_validation['rdto_nextday']

predictions = []
real_values = []

print(len(X_ds)-train)

for i in range(len(X_ds)-train):
    
    grouped_predictions = []

    for x in range(1,lag+1,1):
        
        new_X,new_Y = ts_generator(lag,X_ds,Y_ds)
        
        new_X = new_X[len(new_X)-90:len(new_X)]
        new_Y = new_Y[len(new_Y)-90:len(new_Y)]
        
        X = new_X.copy()
        y = new_Y.copy()
    
        X = X[i:train+i]
        y = y[i:train+i]
        

        X = X.drop(columns=['rdto'])
        
        X_predict = new_X.copy()
        X_predict = X_predict[train+i:train+i+1]
        
        X_rdto = X_predict['rdto']
        X_predict = X_predict.drop(columns=['rdto'])
        
        
        y_real = new_Y.copy()
        y_real = y_real[train+i:train+i+1]
        
        
        model = svm.SVC(C=1, kernel='rbf', gamma='auto')
        model = model.fit(X,y)
        y_pred = model.predict(X_predict)
        
        grouped_predictions.append(y_pred[0])

       # acc = accuracy_score(y_real, y_pred)
       # conf_matrix = confusion_matrix(y_real, y_pred)

        def f(x):
            y = x
            if(x == 0): y=-1 
            else: y=1
            return y

       #y_pred = ([f(x) for x in y_pred])
    print("Grouped Predictions train: ",i," "," = ",sum(grouped_predictions)/lag)
        
    

90
Grouped Predictions train:  0    =  1.0
Grouped Predictions train:  1    =  1.0
Grouped Predictions train:  2    =  1.0
Grouped Predictions train:  3    =  1.0
Grouped Predictions train:  4    =  1.0
Grouped Predictions train:  5    =  1.0
Grouped Predictions train:  6    =  1.0
Grouped Predictions train:  7    =  1.0
Grouped Predictions train:  8    =  1.0
Grouped Predictions train:  9    =  1.0
Grouped Predictions train:  10    =  1.0
Grouped Predictions train:  11    =  1.0
Grouped Predictions train:  12    =  1.0
Grouped Predictions train:  13    =  1.0
Grouped Predictions train:  14    =  1.0
Grouped Predictions train:  15    =  1.0
Grouped Predictions train:  16    =  1.0
Grouped Predictions train:  17    =  1.0
Grouped Predictions train:  18    =  1.0
Grouped Predictions train:  19    =  1.0
Grouped Predictions train:  20    =  1.0
Grouped Predictions train:  21    =  1.0
Grouped Predictions train:  22    =  1.0
Grouped Predictions train:  23    =  1.0
Grouped Predictions tra

ValueError: Found array with 0 sample(s) (shape=(0, 465)) while a minimum of 1 is required.

In [81]:
list_ypred

[array([0.]),
 array([1.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array

In [124]:
X = dataset_validation.copy()
X = X.drop(columns=['rdto_nextday']) #No quitar rdto nominal.
Y = dataset_validation['rdto_nextday']
new_X,new_Y = ts_generator(30,X,Y)

In [125]:
new_X

,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,rdto,t_rdto_1D,t_rdto_5D,t_rdto_10D,...,vader_retweets_vector_lag_30,t_rdto_1D_lag_30,t_rdto_5D_lag_30,t_rdto_10D_lag_30,t_rdto_30D_lag_30,t_rdto_90D_lag_30,volume_5D_lag_30,volume_10D_lag_30,volume_30D_lag_30,volume_90D_lag_30
1073,1,1,1,1,1,1,-1.035319,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1074,1,1,1,1,1,0,-1.457214,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1075,1,1,1,1,1,1,1.082008,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1076,1,1,1,1,1,1,-0.448004,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1077,1,1,1,1,1,1,-1.581249,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,1,1,1,1,1,1,1.047709,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1159,1,1,1,1,1,1,-0.578402,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1160,1,1,1,1,1,1,-1.133534,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1161,1,1,1,1,1,1,0.752050,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [126]:
new_X[len(new_X)-90:len(new_X)]

,tb_likes_vector,tb_comments_vector,tb_retweets_vector,vader_likes_vector,vader_comments_vector,vader_retweets_vector,rdto,t_rdto_1D,t_rdto_5D,t_rdto_10D,...,vader_retweets_vector_lag_30,t_rdto_1D_lag_30,t_rdto_5D_lag_30,t_rdto_10D_lag_30,t_rdto_30D_lag_30,t_rdto_90D_lag_30,volume_5D_lag_30,volume_10D_lag_30,volume_30D_lag_30,volume_90D_lag_30
1073,1,1,1,1,1,1,-1.035319,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1074,1,1,1,1,1,0,-1.457214,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1075,1,1,1,1,1,1,1.082008,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1076,1,1,1,1,1,1,-0.448004,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1077,1,1,1,1,1,1,-1.581249,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,1,1,1,1,1,1,1.047709,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1159,1,1,1,1,1,1,-0.578402,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1160,1,1,1,1,1,1,-1.133534,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1161,1,1,1,1,1,1,0.752050,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [59]:
new_Y[0:30]

1073    0.0
1074    1.0
1075    0.0
1076    0.0
1077    1.0
1078    0.0
1079    0.0
1080    1.0
1081    0.0
1082    0.0
1083    1.0
1084    1.0
1085    1.0
1086    1.0
1087    1.0
1088    0.0
1089    0.0
1090    1.0
1091    0.0
1092    1.0
1093    1.0
1094    1.0
1095    1.0
1096    0.0
1097    0.0
1098    0.0
1099    1.0
1100    1.0
1101    1.0
1102    1.0
Name: rdto_nextday, dtype: float64